# Linear regression tutorial
This notebook will introduce some basic machine learning concepts on the simple but important problem of linear basis function regression. It is inspired by the C.Bishop book "Pattern Recognition and Machine Learning"

We will first include some basic packages and define a few usefull functions that we will use later.

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

def MSE(y,y_est):
    """ Mean squared error function """
    return np.mean((y_est-y)**2)

def pplot(x,y,models=''):
    """ A handy plot function for plotting data and models """
    plt.figure()
    plt.plot(x,y,'o',color=(0.5,0.5,0.5))
    legend=['data']
    for model in models:
        y_est=model[0](x)
        plt.plot(x,y_est,linewidth=5.0)
        legend.append('model %s - MSE:%0.3f' % (model[1],MSE(y,y_est)))
    plt.legend(legend)
    plt.xlabel('x')
    plt.ylabel('y')

## Regression analysis
Regression analysis is commonly addressed general problem for estimating the relationships among variables. Simple practical situation is when there is a set of $N$ observations $x_1,...,x_N$ of the variable $x$ and the corresponding $y_1,...,y_N$ observations of the variable $y$. The regression problem is then in finding the function $f(x)$ that can predict the value of the variable $y$. The selection of the function $f(x)$ is done based on some optimality criterion. A common criterion we will use here is the mean squared error:

$$ L = \frac{1}{N} \sum_{n=1}^N (y_n-f(x_n))^2 $$

As an illustration we first generate some noisy data:

In [ ]:
# generate some data
N = 50 # number of data points
sigma = 0.35 # standard deviation of the additive Gaussain noise
x = np.array([np.linspace(-2, 2, N)]).T # x are on a regularly spaced grid 
f_gt = lambda x: 3 + np.sin(x) + 0.5*np.sin(1.5*x-2) # some function
y = f_gt(x) + sigma*np.random.randn(N,1)

# show the data and the underlying ground truth function
pplot(x,y,[[f_gt,'ground_truth']])

## Linear basis function model
We do not know the real function behind the data in most practical situations. Therefore we need to chose a function. Usually, for practical reasons the choice is first limited to a class of functions. A flexible and general model we consider here is the so called "linear basis function model".

Let $\psi_0(x),...,\psi_{M}(x)$ be $M+1$ fixed non-linear functions of the input variable $x$. Linear combination of the non-linear functions is then:

$$ f(x;{\bf w})=\sum_{m=0}^{M} w_m \psi_m(x) $$

where the non-linear functions $\psi_m( x)$ are known as the "basis functions" and the coefficients ${\bf w}=[w_0,...,w_M]^T$ are the model parameters. 

There are many possible choices for the basis functions. We will use here the polynomial basis:

$$ \psi_m( x)= x^m $$

Other popular choices for basis functions are Gaussian basis, logistic functions, Fourier basis, etc.

It is also common to have one of the functions equal to 1, i.e. $\psi_0(x)=1$, such that the corresponding parameter $w_0$, also known as "bias", describes the global data offset.  
 
The polynomial basis function model is defined in Python code below:

In [ ]:
def psi(x,m):
    """ Polynomial basis function """
    return x**m


def Psi_create_fn(x,M):
    """ Vector of M basis functions """
    return lambda x: np.hstack([psi(x,m) for m in range(M+1)])


def linear_model(w,Psi):
    """ Linear basis function model """
    return Psi.dot(w)

## Least squares model fitting
For a chosen linear basis function model we can find the parameters $\bf w$ that minimize the mean squared error function:

$$ L({\bf w})=\frac{1}{N}\sum_{n=1}^{N} (y_n-f(x_n;{\bf w}))^2 $$

This can be written also in a matrix form:

$$ L({\bf w}) = \frac{1}{N} ({\bf y}- \Psi({\bf x}){\bf w})^T ({\bf y}- \Psi({\bf x}){\bf w})$$

where ${\bf y}=[y_1,...,y_N]^T$  and ${\bf x}=[x_1,...,x_N]^T$ are the observed data points stacked in vectors, and  

$$ \Psi({\bf x})=
\begin{bmatrix}
    \psi_0(x_1)  & \psi_1(x_1) & \dots & \psi_M(x_1) \\
    \psi_0(x_2)  & \psi_1(x_2) & \dots & \psi_M(x_2) \\
    \vdots & \vdots &  \ddots & \vdots \\
    \psi_0(x_N)  & \psi_1(x_N) & \dots & \psi_M(x_N) \\
\end{bmatrix}.
$$

Setting the gradient to zero gives:

$$ \Psi({\bf x})^T\Psi({\bf x}){\bf w}=\Psi({\bf x})^T{\bf y} $$

This can be seen as a set of $M+1$ linear equations that can be solved to find the optimal ${\bf w}_{est}$. One approach is the so called Moore-Penrose pseudo inverse:

$$ {\bf w}_{estimated} = (\Psi({\bf x})^T\Psi({\bf x}))^{-1} \Psi({\bf x})^T{\bf y} $$

While this is a typical textbook method it is also known that it is a particulary poor method in terms of numerical stability. This will be illustrated later.

The code below implements the model fitting procedure and includes also a better linear equations solver based on matrix singular value decomposition.

In [ ]:
def fit_model(x,y,M,method=1,printw=False):
    """ Create an M-th order model and fit it to the data"""
    # create matrix Psi
    Psi_fn = Psi_create_fn(x,M)
    Psi = Psi_fn(x)
    if method==1:
        # Moore-Penrose pseudo-inverse
        w_estimated = np.linalg.inv(Psi.T.dot(Psi)).dot(Psi.T.dot(y))
    else :
        # divide-and-conquer SVD (Lapack xGELSD)
        w_estimated = np.linalg.lstsq(Psi,y,rcond=None)[0]
    # model name
    name = 'M=%d' % M
    # print out the estimated coefficients
    if printw:
        print( name + ',w=' + ','.join('{:.2f}'.format(w) for w in np.nditer(w_estimated)))
    # create the estimated model function
    model = lambda x: linear_model(w_estimated,Psi_fn(x))
    return model, name

## Experiments
Now we have all the important functions implemented and we can try them on the synthetic data and observe some important aspects the regression analysis.

Some suggested experiments to do here are:

1. Try models of order 0,1,2. This should give you the constant, linear and quadratic model. See if you get what is expected.

2. Add more complex models, e.g. M=15. Observe how does it look like. Go above to the data generation step and increase the number of data points to for example N=150. Fit again the M=15 model and observe the results.

3. Further increase the model complexity to for example M=30. Observe the result. Then replace the Moore-Penrose pseudo-inverse with the SVD method.

4. Try even higher dimensional model, e.g. M=50.


In [ ]:
# construct and fit a number of models
models=[fit_model(x,y,M,printw=True) for M in [0,1,2]]

# show the data and the models
pplot(x,y,models)

## Cross validation

The experiments above illustrate probably the central problem in estimation therory, the problem of the model of the "right" complexity. The most common approach for dealing with model selection is the so called "cross validation". One round of cross validation consist typically of dividing the data into "training" and "test" (or "validation") sets. The training data is used to estimate the model paramaters and the test data to evaluate the generalization perfromance of the model. Typically multiple rounds of cross-validation are done with different data partitions. For simplicity we do here just one partition into $75\%$ train and $25\%$ test data.

In [ ]:
indices = np.random.permutation(N)
train_idx, test_idx = indices[:75*N//100], indices[75*N//100:]
x_train, x_test = x[train_idx], x[test_idx]
y_train, y_test = y[train_idx], y[test_idx]

Lets now try a set of models of different complexity $M$ and see how they perfrom on the test and training data.

In [ ]:
error_train=[]
error_test=[]
for M in range(15):
    model,_=fit_model(x_train,y_train,M)
    error_train.append(MSE(y_train,model(x_train)))
    error_test.append(MSE(y_test,model(x_test)))


# show
plt.figure()
plt.plot(error_train)
plt.plot(error_test)
plt.xlabel('M')
plt.legend(['error_train','error_test'])
plt.show()

We can observe that for simple models the training and testing errors are both decresing. After certain model complexity the test error starts increasing while the error on the training data gets always lower. This behaviour is known as "overfitting" the training data.

## Regularization
Another way of dealing with model complexity is to modify the model such that it becomes less prone to overfitting the data. This method is known as model regularization. For our polynomial model a simple heuristic is to add additional constraints such that the weights $w_m$ are kept small and potentially set to zero if they are not needed. One way of achieving this is by adding an additional term to the optimization function:

$$ L({\bf w})=\frac{1}{N}\{\sum_{n=1}^{N} (y_n-f(x_n;{\bf w}))^2 + \beta {\bf w}{\bf w}^T \}$$

Setting the derivative to zero again gives a linear set of equations:

$$\{\beta+\Psi({\bf x})^T\Psi({\bf x})\}{\bf w}=\Psi({\bf x})^T{\bf y} $$

Let us now take a relatively complex model, e.g. M=20, and plot both the regularized and the non-regularized version and observe the difference: 

In [ ]:
def fit_model_regularized(x,y,M,beta,printw=False):
    """ Create an M-th order model and fit it to the data"""
    Psi_fn=Psi_create_fn(x,M)
    Psi=Psi_fn(x)
    w_estimated = np.linalg.lstsq(beta*np.eye(M+1)+Psi.T.dot(Psi),Psi.T.dot(y),rcond=None)[0]
    # model name
    name = 'M=%d(beta=%0.2f)' % (M,beta)
    # print out the estimated coefficients
    if printw:
        print( name + ',w=' + ','.join('{:.2f}'.format(w) for w in np.nditer(w_estimated)))
    # create the estimated model function
    model = lambda x: linear_model(w_estimated,Psi_fn(x))
    return model, name
    

pplot(x,y,[fit_model(x,y,20,printw=True),fit_model_regularized(x,y,20,0.9,printw=True)])


## Summary

The most common task in machine learning is fitting a model to some data. Besides the optimization procedure the central problem in machine learning is selecting the right model. This tutorial gives some hand-on experience on a simple linear basis function model. Main concepts and approaches are illistrated. 